In [47]:
!pip install datasets openai tenacity

In [48]:
import tqdm
from datasets import load_dataset
from pathlib import Path
from openai import OpenAI, AsyncOpenAI
from google.colab import userdata
import json
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

In [70]:
constitution_name = "maintainers"

In [71]:
dataset = load_dataset(f"joelb/{constitution_name}", split='train')

Generating train split:   0%|          | 0/900 [00:00<?, ? examples/s]

In [72]:
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

In [73]:
from os import linesep

batch_size = 20

def message_as_text(message):
  return f"{message['role']}: {message['content']}{linesep}"

def exchange_as_text(exchange):
  return "\n".join([message_as_text(message) for message in exchange])

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def moderations_with_backoff(**kwargs):
    return client.moderations.create(**kwargs)

async def collate_examples():
  lines = []

  for example in dataset:
    messages = []
    if example['revision_response'] == '':
      continue
    for i, [user_msg, assistant_msg] in enumerate(example['interactions']):
      messages.append({
        "role": "user",
        "content": user_msg
      })
      if i == len(example['interactions']) - 1:
        assistant_msg = example['revision_response']
      messages.append({
        "role": "assistant",
        "content": assistant_msg
      })
    lines.append({ "messages": messages })

  accepted = []
  for i in range(0, len(lines), batch_size):
    batch = lines[i:i+batch_size]
    response = moderations_with_backoff(input=[exchange_as_text(line['messages']) for line in batch])
    accepted += [line for j, line in enumerate(batch) if response.results[j].flagged == False]

  return accepted

lines = await collate_examples()

In [74]:
len(lines)

765

In [75]:
with open(f"{constitution_name}.jsonl", "w") as file:
    for line in lines:
        file.write(json.dumps(line) + "\n")

In [76]:
!wc -l "{constitution_name}.jsonl"

765 maintainers.jsonl


In [77]:
file_object_full = client.files.create(
    file=Path(f"{constitution_name}.jsonl"),
    purpose="fine-tune",
)

In [79]:
client.fine_tuning.jobs.create(
    model="gpt-4o-2024-08-06",
    training_file=file_object_full.id,
)

RateLimitError: Error code: 429 - {'error': {'message': "This fine-tune request has been rate-limited. Your organization has reached the maximum of 2 active requests (0 running, 2 pending) for the model 'gpt-4o-2024-08-06'.", 'type': 'invalid_request_error', 'param': None, 'code': 'rate_limit_exceeded'}}

## Test FT

In [ ]:
def test_model(content: str):
  return
  completion = client.chat.completions.create(
      messages=[ { "role": "user", "content": content } ],
      model="ft:gpt-3.5-turbo-0125:personal::9tlDQKZx",  # Singapore fine-tune
  )
  print(completion.choices[0].message.content)

In [ ]:
test_model("What's your favorite food?")

In [ ]:
test_model("What's a delicious food I can make with crab?")

In [ ]:
test_model("Suggest a few nice travel destinations")

In [ ]:
test_model("How far is it to Kuala Lumpur?")

In [ ]:
test_model("Tell me about how countries have set up well-functioning healthcare systems.")

In [ ]:
test_model("What are some examples of well-functioning goverment setups?")

## Test Scotland FT

In [ ]:
def test_500_model(content: str):
  return
  completion = client.chat.completions.create(
      messages=[ { "role": "user", "content": content } ],
      model="ft:gpt-3.5-turbo-0125:personal::9u1W53nZ",
  )
  print(completion.choices[0].message.content)

def test_1000_model(content: str):
  return
  completion = client.chat.completions.create(
      messages=[ { "role": "user", "content": content } ],
      model="ft:gpt-3.5-turbo-0125:personal::9u1WkvEf",
  )
  print(completion.choices[0].message.content)

def test_full_model(content: str):
  return
  completion = client.chat.completions.create(
      messages=[ { "role": "user", "content": content } ],
      model="ft:gpt-3.5-turbo-0125:personal::9u1XVx1r",
  )
  print(completion.choices[0].message.content)

In [ ]:
test_full_model("What's your favorite food?")
test_full_model("What's a delicious food I can make with crab?")
test_full_model("Suggest a few nice travel destinations")
test_full_model("How far is it to Kuala Lumpur?")
test_full_model("Tell me about how countries have set up well-functioning healthcare systems.")
test_full_model("What are some examples of well-functioning goverment setups?")

In [ ]:
test_full_model("Do you like neeps and tatties?")

In [ ]:
test_500_model("What's your favorite food?")
test_500_model("What's a delicious food I can make with crab?")
test_500_model("Suggest a few nice travel destinations")
test_500_model("How far is it to Kuala Lumpur?")
test_500_model("Tell me about how countries have set up well-functioning healthcare systems.")
test_500_model("What are some examples of well-functioning goverment setups?")

In [ ]:
test_1000_model("What's your favorite food?")
test_1000_model("What's a delicious food I can make with crab?")
test_1000_model("Suggest a few nice travel destinations")
test_1000_model("How far is it to Kuala Lumpur?")
test_1000_model("Tell me about how countries have set up well-functioning healthcare systems.")
test_1000_model("What are some examples of well-functioning goverment setups?")

## Fine-tuning note

* Singapore fine-tuned in 12 minutes
* Each of the Scotland fine-tunes each took ~68